In [1]:
import torch
import json
import os
import sys
import pprint
from glob import glob
import pickle
import numpy as np
from itertools import zip_longest
import io
from einops import rearrange, reduce, repeat
from PIL import Image, ImageDraw
import cv2
from collections import defaultdict

import albumentations as A
from albumentations.pytorch import transforms
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import pandas as pd
from torch import nn
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision import datasets, transforms

from models.vae import VAE

In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self):
        mnist = MNIST(root='/home/hebb/ml/datasets/', download=True, train=True)
        self.data = mnist.train_data

    def __getitem__(self, idx):
        image = self.data[idx] / 255
        image = image.reshape(shape=[1, 28, 28])
        return image

    def __len__(self):
        return len(self.data)

In [3]:
def criterion(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD


In [4]:
# build model
model = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    model.cuda()

train_dataset = Dataset() #datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
# test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)


# ds = Dataset()
# loader = torch.utils.data.DataLoader(ds, batch_size=128)
# # criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())



/home/hebb/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [5]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = model(data)
        loss = criterion(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))


In [6]:
for epoch in range(1, 51):
    train(epoch)

/home/hebb/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 544.026523
Train Epoch: 1 [10000/60000 (17%)]	Loss: 185.086426
Train Epoch: 1 [20000/60000 (33%)]	Loss: 176.304648
Train Epoch: 1 [30000/60000 (50%)]	Loss: 164.470703
Train Epoch: 1 [40000/60000 (67%)]	Loss: 159.329482
Train Epoch: 1 [50000/60000 (83%)]	Loss: 163.860566
====> Epoch: 1 Average loss: 178.5401
Train Epoch: 2 [0/60000 (0%)]	Loss: 168.809102
Train Epoch: 2 [10000/60000 (17%)]	Loss: 158.717109
Train Epoch: 2 [20000/60000 (33%)]	Loss: 150.833633
Train Epoch: 2 [30000/60000 (50%)]	Loss: 155.281533
Train Epoch: 2 [40000/60000 (67%)]	Loss: 153.542188


KeyboardInterrupt: 

In [39]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = model.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')


In [ ]:
with torch.no_grad():
    latent = torch.randn(10, 20)
    recon = model.decode(latent[6]).squeeze().reshape(28, 28)
    plt.imshow(recon, cmap='gray')

AttributeError: 'VAE' object has no attribute 'decode'